In [12]:
import pandas as pd

df = pd.read_csv('sinais_vitais_com_label_treino.txt', sep=',', header=0)
df

,id,pSist,pDiast,qPA,pulso,frequencia,gravidade,classe
0,1,18.907009,12.220855,8.733333,134.454047,17.972046,34.192304,2
1,2,14.698109,13.586879,8.614814,86.302237,4.168373,40.000000,2
2,3,16.428095,1.904802,-4.333333,138.880782,2.721256,27.122605,2
3,4,14.616501,13.700638,8.602531,51.356913,18.061923,47.188993,2
4,5,10.955766,9.485389,-0.000000,1.951730,14.033748,68.333333,3
...,...,...,...,...,...,...,...,...
795,796,11.478356,0.769978,-4.427394,102.221274,0.761861,43.185667,2
796,797,8.353375,1.093279,-5.561934,73.366682,9.924723,43.067405,2
797,798,3.232797,1.327912,0.000000,147.895901,0.303480,40.000000,2
798,799,13.457415,11.975263,7.818706,104.948033,10.421636,37.262834,2


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

df = pd.read_csv('sinais_vitais_com_label_treino.txt', sep=',', header=0)
X = df.iloc[:, 3:6]
y = df.iloc[:, 6]

model1 = MLPRegressor(hidden_layer_sizes=(50,), activation='relu', solver='adam', learning_rate_init=0.01, max_iter=200)
model2 = MLPRegressor(hidden_layer_sizes=(100,), activation='logistic', solver='sgd', learning_rate_init=0.01, max_iter=200)
model3 = MLPRegressor(hidden_layer_sizes=(50, 50), activation='tanh', solver='adam', learning_rate_init=0.01, max_iter=200)

models = [model1, model2, model3]
for i, model in enumerate(models, start=1):
    model.fit(X, y)
    predictions = model.predict(X)
    rmse = np.sqrt(mean_squared_error(y, predictions))
    print(f'Model {i} RMSE: {rmse}')

best_model = model1

blind_test_data = pd.read_csv('sinais_vitais_validacao.txt', sep=',', header=0)
X_blind_test = blind_test_data.iloc[:, 3:6]

blind_test_predictions = best_model.predict(X_blind_test)

pd.DataFrame(blind_test_predictions).to_csv('predictions.csv', index=False, header=False)

C:\Users\henri\PycharmProjects\VictimSim\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Model 1 RMSE: 10.943130536160966
Model 2 RMSE: 16.153811216406016
Model 3 RMSE: 8.04822531291944


C:\Users\henri\PycharmProjects\VictimSim\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [12]:
actual_values = df['gravidade'][:len(blind_test_predictions)]

differences = actual_values - blind_test_predictions

mean_difference = differences.mean()
std_difference = differences.std()

print(f'Mean difference: {mean_difference}')
print(f'Standard deviation of differences: {std_difference}')

Mean difference: 1.6050085464421942
Standard deviation of differences: 9.499476487662104


In [24]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

hidden_layer_sizes_options = [(50,), (100,), (50, 50), (100, 100), (50, 50, 50), (100, 100, 100), (50, 50, 50, 50), (100, 100, 100, 100)]
activation_functions = ['identity', 'logistic', 'tanh', 'relu']
solvers = ['lbfgs', 'sgd', 'adam']
learning_rates = ['constant', 'invscaling', 'adaptive']
max_iterations = [100, 200, 300]

# learning_rate_inits = np.arange(0.001, 0.051, 0.001)
# fixed learning_rate_init of 0.01
learning_rate_inits = [0.01]

configurations = []
for hidden_layer_sizes in hidden_layer_sizes_options:
    for activation in activation_functions:
        for solver in solvers:
            for learning_rate in learning_rates:
                for max_iter in max_iterations:
                    for learning_rate_init in learning_rate_inits:
                        if solver == 'sgd':
                            configurations.append({'hidden_layer_sizes': hidden_layer_sizes, 'activation': activation, 'solver': solver, 'learning_rate': learning_rate, 'max_iter': max_iter, 'learning_rate_init': learning_rate_init})
                        else:
                            configurations.append({'hidden_layer_sizes': hidden_layer_sizes, 'activation': activation, 'solver': solver, 'max_iter': max_iter, 'learning_rate_init': learning_rate_init})

best_rmse = np.inf
best_model = None
best_config = None

for config in configurations:
    try:
        model = MLPRegressor(**config)
        model.fit(X, y)
    except ValueError:
        continue
    
    predictions = model.predict(X)
    rmse = np.sqrt(mean_squared_error(y, predictions))
    
    # Print the RMSE for this model
    # print(f'Model with {config} RMSE: {rmse}')
    
    if rmse < best_rmse:
        best_rmse = rmse
        best_model = model
        best_config = config

print(f'Best model configuration: {best_config}')
print(f'Best model RMSE: {best_rmse}')

C:\Users\henri\PycharmProjects\VictimSim\venv\Lib\site-packages\sklearn\neural_network\_base.py:173: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
C:\Users\henri\PycharmProjects\VictimSim\venv\Lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\henri\PycharmProjects\VictimSim\venv\Lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\henri\PycharmProjects\VictimSim\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\henri\PycharmProjects\VictimSim\venv\Lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\henri\PycharmProjects\VictimSim\venv\Lib\site-packages\sklearn\utils\extmat

Best model configuration: {'hidden_layer_sizes': (50, 50), 'activation': 'tanh', 'solver': 'adam', 'max_iter': 300, 'learning_rate_init': 0.01}
Best model RMSE: 5.250459384138478


In [11]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import numpy as np

# Define the hyperparameters
param_grid = {
    'hidden_layer_sizes': [(100, 100)],
    'activation': ['tanh'],
    'solver': ['adam'],
    'learning_rate': ['adaptive'],
    'max_iter': [500, 600, 700, 800, 900, 1000],
    # 'max_iter': [500],
    # 'learning_rate_init': np.arange(0.001, 0.01, 0.001)
    'learning_rate_init': [0.002],
    'batch_size': [32, 64, 128, 256, 512, 1024],
}

# Create a MLPRegressor
mlp = MLPRegressor()

# Create the GridSearchCV object
grid_search = GridSearchCV(mlp, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the GridSearchCV object
grid_search.fit(X, y)

# Get the best model
best_model = grid_search.best_estimator_

# Calculate the RMSE of the best model
predictions = best_model.predict(X)
rmse = np.sqrt(mean_squared_error(y, predictions))

print(f'Best model configuration: {grid_search.best_params_}')
print(f'Best model RMSE: {rmse}')

Best model configuration: {'activation': 'tanh', 'batch_size': 32, 'hidden_layer_sizes': (100, 100), 'learning_rate': 'adaptive', 'learning_rate_init': 0.002, 'max_iter': 800, 'solver': 'adam'}
Best model RMSE: 1.9451869855860497


In [12]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import numpy as np

# Define the hyperparameters
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)], (50, 50, 50), (100, 100, 100), (50, 50, 50, 50), (100, 100, 100, 100)],
    'activation': ['tanh'],
    'solver': ['adam'],
    'max_iter': [500, 600, 700],
    'learning_rate_init': np.arange(0.001, 0.01, 0.001)
}

mlp = MLPRegressor()

grid_search = GridSearchCV(mlp, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(X, y)

best_model = grid_search.best_estimator_

# Calculate the RMSE of the best model
predictions = best_model.predict(X)
rmse = np.sqrt(mean_squared_error(y, predictions))

print(f'Best model configuration: {grid_search.best_params_}')
print(f'Best model RMSE: {rmse}')

SyntaxError: closing parenthesis ']' does not match opening parenthesis '{' on line 7 (2777465128.py, line 8)